#### Wednesday, October 30, 2024

[Build a Simple LLM Application with LCEL](https://python.langchain.com/docs/tutorials/llm_chain/)

mamba activate langchain

Ok. Nice! This works with LMStudio using hermes-3-llama-3.1-8b.

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [2]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4")
# model = ChatOpenAI(model="gpt-3.5-turbo")

model = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

In [4]:
model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 23, 'total_tokens': 26, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'hermes-3-llama-3.1-8b', 'system_fingerprint': 'hermes-3-llama-3.1-8b', 'finish_reason': 'stop', 'logprobs': None}, id='run-66aab8f1-61a9-4111-81fd-5ad48e405d5f-0', usage_metadata={'input_tokens': 23, 'output_tokens': 3, 'total_tokens': 26, 'input_token_details': {}, 'output_token_details': {}})

#### OutputParsers

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [6]:
result = model.invoke(messages)

In [7]:
parser.invoke(result)

'Ciao!'

In [8]:
chain = model | parser

In [9]:
chain.invoke(messages)

'Ciao!'

#### Prompt Templates

(Do the next few cells call OpenAI or LMStudio?)

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
system_template = "Translate the following into {language}:"

In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [13]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [14]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

#### Chaining together components with LECL

In [15]:
chain = prompt_template | model | parser

In [16]:
chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

#### Serving with LangServe